# Importing Dependencies

In [431]:
from google.colab import drive
import pandas as pd
import numpy as np
import scipy.spatial
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Methods 

## FKNN

In [432]:
import scipy.spatial
from collections import Counter
from operator import itemgetter

class FKNN:
    def __init__(self, k):
      self.k = k
        
    def fit(self, X, y):
      self.X_train = X
      self.y_train = y

    def getDistance(self, X1, X2):
      return scipy.spatial.distance.euclidean(X1, X2)

    def fuzzy(self, d, m):
      closestPoint = [ d[k][1] for k in range(len(d))]
      classes = np.unique(self.y_train[closestPoint])
      arrMembership = []
      for cls in classes:
        atas = 0
        bawah = 0
        for close in d: 
          if(close[0] != 0):
            if(cls == self.y_train[close[1]]):
              atas += np.power((1/close[0]), (2/(m-1)))
            else:
              atas += np.power((0/close[0]), (2/(m-1)))
            bawah +=  np.power((1/close[0]), (2/(m-1)))
          else:
            atas  += 0
            bawah  += 1
        arrMembership.append([atas/bawah, cls])
      return arrMembership

    def predict(self, X_test):
      final_output = []
      for i in range(len(X_test)):
          d = []
          votes = []
          for j in range(len(X_train)):
              dist = self.getDistance(X_train[j] , X_test[i])
              d.append([dist, j])
          d.sort()
          d = d[0:self.k]
          membership = self.fuzzy(d, 2)
          predicted_class = sorted(membership, key=itemgetter(0), reverse=True)
          final_output.append(predicted_class[0][1])    
      return final_output
    
    def score(self, X_test, y_test):
      predictions = self.predict(X_test)
      value = 0
      for i in range(len(y_test)):
        if(predictions[i] == y_test[i]):
          value += 1
      return value / len(y_test)

## BM-FKNN

In [433]:
import scipy.spatial
from collections import Counter
from operator import itemgetter

class BM_FKNN:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def getDistance(self, X1, X2):
        return scipy.spatial.distance.euclidean(X1, X2)

    def getFirstDistance(self, X_train, X_test):
      distance = []
      for i in range(len(X_train)):
        dist = scipy.spatial.distance.euclidean(X_train[i] , X_test)
        distance.append([i, dist, self.y_train[i]])
      return distance
      
    def nearestPoint(self, X_test):
      allPoint = [ i for i in range(len(X_test))]
      distance = self.getFirstDistance(self.X_train, X_test)   
      distance_sorted = sorted(distance, key=itemgetter(1))
      closest = distance_sorted[0:self.k]
      closestPoint = [ i[0] for i in closest]
      anothersPoint = np.setdiff1d(allPoint, closestPoint)
      return closestPoint, anothersPoint

    def bonferroniMean(self, c, closestPoint, p, q): 
      arrInner = [self.X_train[e] for e in closestPoint if(self.y_train[e] != c)]
      arrOuter = [self.X_train[q] for q in closestPoint if(self.y_train[q] == c)]
      n = len(closestPoint)
      if(n > 1):
        inner = [(sum(np.power(x, q)))/n for x in zip(*arrInner)]
        outer = [(sum(np.power(x, p)))/(n-1) for x in zip(*arrOuter)]
      else:
        inner = arrInner[0].copy()
        outer = arrOuter[0].copy()
      Br = [ np.power((inner[i]*outer[i]), (1/(p+q)) ) for i in range(len(inner))]
      return Br
    
    def fuzzy(self, arrBr, closestPoint, m):
      arrMembership = []
      for localMean in arrBr:
        atas = 0
        bawah = 0
        for r in (closestPoint): 
          if(localMean[1] == self.y_train[r]):
            atas += np.power((1/localMean[0]), (2/(m-1)))
          else:
            atas += np.power((0/localMean[0]), (2/(m-1)))
          bawah +=  np.power((1/localMean[0]), (2/(m-1)))
        arrMembership.append([atas/bawah, localMean[1]])
      return arrMembership

    def predict(self, X_test, p, q, m):
      final_output = []
      for i in range(len(X_test)):
        localMean = []
        closestPoint, anothersPoint = self.nearestPoint(X_test[i])
        classes = np.unique(self.y_train[closestPoint])
        if(len(classes) == 1):
          final_output.append(classes[0]) 
        else:
          arrBr = []
          for j in classes:
            Br = self.bonferroniMean(j, closestPoint, p, q)
            distBr = self.getDistance(X_test[i], Br)
            arrBr.append([distBr, j])
          membership = self.fuzzy(arrBr, closestPoint, m )
          predicted_class = sorted(membership, key=itemgetter(0), reverse=True)
          final_output.append(predicted_class[0][1])
      return final_output
    
    def score(self, X_test, y_test, p, q, m):
      predictions = self.predict(X_test, p, q, m)
      value = 0
      for i in range(len(y_test)):
        if(predictions[i] == y_test[i]):
          value += 1
      return value / len(y_test)
     

# Examples

## Wine


In [434]:
train_path = r'wine.xlsx'
data_train = pd.read_excel(train_path)
data_train.head()

,Alcohol,MalicAcid,Ash,AlcalinityOfAsh,Magnesium,TotalPhenols,flavanoids,NonflavanoidsPhenols,Proanthocyanins,ColorIntensity,Hue,OD280/OD315,Proline,Class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [435]:

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap

countClass = data_train['Class'].value_counts().reset_index()
countClass.columns = ['Class', 'count']
print(countClass)

fig = px.pie(
    countClass, 
    values='count', 
    names="Class", 
    title='Class Distribution', 
    width=700, 
    height=500
)

fig.show()

   Class  count
0      2     71
1      1     59
2      3     48


In [436]:
np.unique(np.array(data_train['Class']))

array([1, 2, 3])

In [437]:
label_train = data_train.iloc[:,-1].to_numpy()
fitur_train = data_train.iloc[:,:13].to_numpy()


In [438]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(fitur_train)
fitur_train_normalize = scaler.transform(fitur_train)

### BM-FKNN

In [439]:
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)

rataBMFKNN = []
for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  bmfknn = BM_FKNN(25)
  bmfknn.fit(X_train, y_train)
  prediction = bmfknn.score(X_test, y_test, 2, 2, 1.5)
  rataBMFKNN.append(prediction)
  
print('Mean Accuracy: ', np.mean(rataBMFKNN))

Mean Accuracy:  0.961111111111111


### FKNN

In [440]:
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)

accuracyFKNN = []

for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  fknn = FKNN(25)
  fknn.fit(X_train, y_train)
  prediction = fknn.score(X_test, y_test)
  accuracyFKNN.append(prediction)

print('Mean Accuracy: ', np.mean(accuracyFKNN))

Mean Accuracy:  0.9666666666666666


### KNN

In [441]:
from sklearn.neighbors import KNeighborsClassifier
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)
rata = []

for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  neigh = KNeighborsClassifier(n_neighbors=25)
  neigh.fit(X_train, y_train)
  prediction = neigh.score(X_test, y_test)
  rata.append(prediction)

print('Mean Accuracy: ', np.mean(rata))

Mean Accuracy:  0.9666666666666666


## Ionosphere

In [442]:
train_path = r'ionosphere.xlsx'
data_train = pd.read_excel(train_path)
data_train.head()
     

,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,30,31,32,33,34,Class
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [443]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap

countClass = data_train['Class'].value_counts().reset_index()
countClass.columns = ['Class', 'count']
print(countClass)

fig = px.pie(
    countClass, 
    values='count', 
    names="Class", 
    title='Class Distribution', 
    width=700, 
    height=500
)

fig.show()

  Class  count
0     g    225
1     b    126


In [444]:
np.unique(np.array(data_train['Class']))

array(['b', 'g'], dtype=object)

### BM-FKNN

In [445]:
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)

rataBMFKNN = []
for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  bmfknn = BM_FKNN(7)
  bmfknn.fit(X_train, y_train)
  prediction = bmfknn.score(X_test, y_test, 1, 0, 1.5)
  rataBMFKNN.append(prediction)
  
print('Mean Accuracy: ', np.mean(rataBMFKNN))

Mean Accuracy:  0.9555555555555555


### FKNN

In [446]:
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)

accuracyFKNN = []
for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  fknn = FKNN(7)
  fknn.fit(X_train, y_train)
  prediction = fknn.score(X_test, y_test)
  accuracyFKNN.append(prediction)

print('Mean Accuracy: ', np.mean(accuracyFKNN))

Mean Accuracy:  0.9666666666666666


### KNN

In [447]:
from sklearn.neighbors import KNeighborsClassifier
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)
rata = []

for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  neigh = KNeighborsClassifier(n_neighbors=7)
  neigh.fit(X_train, y_train)
  prediction = neigh.score(X_test, y_test)
  rata.append(prediction)

print('Mean Accuracy: ', np.mean(rata))

Mean Accuracy:  0.9555555555555555


## Vehicle

In [448]:
train_path = r'vehicle.xlsx'
data_train = pd.read_excel(train_path)
data_train.head()

,Compactness,Circularity,Distance_circularity,Radius_ratio,Praxis_aspect_ratio,Max_length_aspect_ratio,Scatter_ratio,Elongatedness,Praxis_rectangular,Length_rectangular,Major_variance,Minor_variance,Gyration_radius,Major_skewness,Minor_skewness,Minor_kurtosis,Major_kurtosis,Hollows_ratio,Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus


In [449]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap

countClass = data_train['Class'].value_counts().reset_index()
countClass.columns = ['Class', 'count']
print(countClass)

fig = px.pie(
    countClass, 
    values='count', 
    names="Class", 
    title='Class Distribution', 
    width=700, 
    height=500
)

fig.show()

   Class  count
0   bus     218
1   saab    217
2   opel    212
3   van     199


In [450]:
np.unique(np.array(data_train['Class']))

array([' bus ', ' opel', ' saab', ' van '], dtype=object)

In [451]:
label_train = data_train.iloc[:,-1].to_numpy()
fitur_train = data_train.iloc[:,:18].to_numpy()

In [452]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(fitur_train)
fitur_train_normalize = scaler.transform(fitur_train)

### BM-FKNN

In [453]:
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)

rataBMFKNN = []
for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  bmfknn = BM_FKNN(4)
  bmfknn.fit(X_train, y_train)
  prediction = bmfknn.score(X_test, y_test, 3, 1, 2)
  rataBMFKNN.append(prediction)
  
print('Mean Accuracy: ', np.mean(rataBMFKNN))

Mean Accuracy:  0.6891736694677871


### FKNN

In [454]:
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)

accuracyFKNN = []

for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  fknn = FKNN(4)
  fknn.fit(X_train, y_train)
  prediction = fknn.score(X_test, y_test)
  accuracyFKNN.append(prediction)

print('Mean Accuracy: ', np.mean(accuracyFKNN))


Mean Accuracy:  0.7151680672268907


### KNN

In [455]:
from sklearn.neighbors import KNeighborsClassifier
kf = KFold(n_splits=10, random_state=1, shuffle=True) 
kf.get_n_splits(fitur_train_normalize)
rata = []

for train_index, test_index in kf.split(fitur_train_normalize):
  X_train, X_test = fitur_train_normalize[train_index], fitur_train_normalize[test_index]
  y_train, y_test = label_train[train_index], label_train[test_index]

  neigh = KNeighborsClassifier(n_neighbors=4)
  neigh.fit(X_train, y_train)
  prediction = neigh.score(X_test, y_test)
  rata.append(prediction)

print('Mean Accuracy: ', np.mean(rata))

Mean Accuracy:  0.6891736694677871
